<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW6_%E8%AA%B2%E8%A1%A8%E6%9F%A5%E8%A9%A2%E8%88%87%E6%8F%90%E9%86%92%E7%B4%A0%E6%9D%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install pdfplumber pandas gspread google-colab google-genai

import pdfplumber
import pandas as pd
import numpy as np
import io
import gspread
from google.colab import auth
from google.auth import default

FILE_NAME = '114-1課表.pdf'

In [26]:
try:
    with pdfplumber.open(FILE_NAME) as pdf:
        page = pdf.pages[0]
        tables = page.extract_tables()

        if not tables:
            print("❌ 警告：pdfplumber 未能自動識別出表格。")
            df_schedule = None
        else:
            table_data = tables[0]

            # 清理資料：移除空行
            cleaned_data = [
                [cell.replace('\n', ' ').strip() if cell else '' for cell in row]
                for row in table_data if any(cell and cell.strip() for cell in row)
            ]

            columns = ['課程中文名稱', '上課教室', '星期', '節次', '上課時間']
            df_schedule = pd.DataFrame(cleaned_data[1:], columns=columns)

            # 關鍵清洗：使用前一個非空值填充「星期」欄位
            df_schedule['星期'] = df_schedule['星期'].replace('', np.nan).fillna(method='ffill')

            # 補齊作業所需的「本週重點」欄位
            if '本週重點' not in df_schedule.columns:
                df_schedule['本週重點'] = ''

            print("✅ 成功使用 pdfplumber 自動擷取表格。")

except FileNotFoundError:
    print(f"❌ 錯誤：找不到檔案 {FILE_NAME}。請確認您已上傳檔案。")
    df_schedule = None
except Exception as e:
    print(f"❌ 處理 PDF 時發生錯誤: {e}")
    df_schedule = None


# --- 1B: Google Sheet 授權與首次回寫 ---

if df_schedule is not None and not df_schedule.empty:

    # 授權與連線
    try:
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        print("✅ Google Sheet 授權成功。")
    except Exception as e:
        print(f"❌ Google Sheet 授權失敗: {e}")
        raise RuntimeError("無法連線至 Google 服務。")

    SHEET_TITLE = '作業六：課表提醒結果'
    WORKSHEET_TITLE = '自動擷取課表_請填寫本週重點'

    try:
        sh_result = gc.open(SHEET_TITLE)
    except gspread.WorksheetNotFound:
        sh_result = gc.create(SHEET_TITLE) # 建立新的 Sheet

    try:
        worksheet = sh_result.worksheet(WORKSHEET_TITLE)
    except gspread.WorksheetNotFound:
        worksheet = sh_result.add_worksheet(title=WORKSHEET_TITLE, rows="100", cols="20")

    # 將 DataFrame 寫入 Google Sheet (覆蓋操作，只執行一次)
    worksheet.clear()
    worksheet.update([df_schedule.columns.values.tolist()] + df_schedule.values.tolist())

    print(f"\n✅ 資料已首次回寫至 Google Sheet 的分頁: {WORKSHEET_TITLE}")
    print(f"🔗 Google Sheet 連結: {sh_result.url}")
else:
    print("❌ 由於課表資料無效，無法執行 Google Sheet 寫入。")

/tmp/ipython-input-3103532623.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_schedule['星期'] = df_schedule['星期'].replace('', np.nan).fillna(method='ffill')


✅ 成功使用 pdfplumber 自動擷取表格。
✅ Google Sheet 授權成功。

✅ 資料已首次回寫至 Google Sheet 的分頁: 自動擷取課表_請填寫本週重點
🔗 Google Sheet 連結: https://docs.google.com/spreadsheets/d/17n4A3u3wH7CD08Mfqd4PYQkaYoJshuYG93h1eICD_oE


In [27]:
import pandas as pd
import gspread
from google.colab import auth, userdata
from google.auth import default
from google import genai
import os
import datetime

# --- AI Client 初始化 (從 Colab Secrets 載入) ---
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
    client = genai.Client()
except Exception as e:
    print(f"❌ Gemini Client 初始化失敗：{e}。請檢查 Colab Secrets 中的 'GEMINI_API_KEY'。")
    client = None

# --- 全域變數定義 ---
gc = None
sh_result = None
WORKSHEET_TITLE = '自動擷取課表_請填寫本週重點'
COL_COURSE = '課程中文名稱'
COL_WEEKDAY = '星期'
COL_NOTES = '本週重點'
COL_ROOM = '上課教室'
COL_TIME = '上課時間'

# --- 核心輔助函式 ---

def initialize_gspread_and_data():
    """初始化 Google Sheet 連線並讀取資料"""
    global gc, sh_result

    try:
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
    except Exception as e:
        return f"❌ Google Sheet 授權失敗：{e}", None

    SHEET_TITLE = '作業六：課表提醒結果'
    try:
        sh_result = gc.open(SHEET_TITLE)
    except gspread.WorksheetNotFound:
        return f"❌ 找不到目標 Google Sheet: {SHEET_TITLE}", None

    try:
        worksheet = sh_result.worksheet(WORKSHEET_TITLE)
        df = pd.DataFrame(worksheet.get_all_records())

        required_cols = [COL_COURSE, COL_WEEKDAY, COL_NOTES, COL_ROOM, COL_TIME]
        if not all(col in df.columns for col in required_cols):
             return f"❌ 資料欄位不完整。缺少欄位：{list(set(required_cols) - set(df.columns))}", None

        return "✅ 連線成功，資料已載入。", df
    except gspread.WorksheetNotFound:
        return f"❌ 找不到工作表: {WORKSHEET_TITLE}", None
    except Exception as e:
        return f"❌ 讀取資料失敗: {e}", None

# --- Gradio 函式 1: 更新/清空提醒 (實現覆蓋刷新邏輯) ---

def update_reminder_and_writeback(course_name, new_note):
    """手動更新單一課程的「本週重點」並回寫到 Google Sheet"""

    status, df_schedule = initialize_gspread_and_data()

    # 讀取失敗時，將狀態訊息作為唯一輸出
    if df_schedule is None:
        return status, []

    # 找到要更新的課程行
    idx = df_schedule[df_schedule[COL_COURSE] == course_name].index

    if idx.empty:
        return f"❌ 找不到課程: {course_name}", []

    # 更新 DataFrame (可能有多行)
    # new_note 可以是空字串 ('')，實現清空/覆蓋刷新
    df_schedule.loc[idx, COL_NOTES] = new_note

    # 將更新後的 DataFrame 寫回 Google Sheet (覆蓋整個工作表)
    try:
        # **【安全防護】** 只有當 DataFrame 有資料行時才執行覆蓋！
        if df_schedule.empty:
            return f"❌ 警告：DataFrame 為空，已取消回寫操作以防資料遺失。", df_schedule[COL_COURSE].unique().tolist()

        worksheet = sh_result.worksheet(WORKSHEET_TITLE)
        worksheet.clear()
        worksheet.update([df_schedule.columns.values.tolist()] + df_schedule.values.tolist())

        course_list = df_schedule[COL_COURSE].unique().tolist()

        action_desc = "清空" if new_note == '' else f"更新為：『{new_note}』"

        return f"✅ 已成功 {action_desc} **{course_name}** 的本週重點並回寫至 Google Sheet。", course_list

    except Exception as e:
        return f"❌ 回寫 Google Sheet 失敗: {e}", []


# --- Gradio 函式 2: 查詢課程與 AI 提醒生成 ---

def generate_daily_reminder(target_weekday):
    """查詢指定日期的課程並利用 AI 生成行前提醒"""

    status, df_schedule = initialize_gspread_and_data()
    if df_schedule is None:
        return status, "", "", "", ""

    daily_schedule = df_schedule[df_schedule[COL_WEEKDAY] == target_weekday].copy()

    if daily_schedule.empty:
        return f"✅ {target_weekday} 沒有排課！太棒了！", "", "", "", ""

    # 格式化當日課程清單
    courses_details = []
    for index, row in daily_schedule.iterrows():
        details = (
            f"**{row[COL_COURSE]}** ({row[COL_TIME]} / {row[COL_ROOM].strip()}) "
            f"[本週重點: {row[COL_NOTES]}]"
        )
        courses_details.append(details)

    formatted_schedule = "\n- " + "\n- ".join(courses_details)
    all_notes = daily_schedule[COL_NOTES].str.cat(sep="；")

    # 準備 AI 提示詞
    ai_prompt = (
        f"您是一位專業的課表助理。根據以下 {target_weekday} 的課程和本週重點，"
        f"請您將所有重點彙整成**一句**精簡、友善的『行前提醒』，"
        f"內容應涵蓋**攜帶品、先讀章節或最重要的單一任務**。文字需以繁體中文輸出，長度不超過50字。 "
        f"本週重點內容：『{all_notes}』"
    )

    # 呼叫 Gemini API
    if client:
        try:
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=ai_prompt,
                config=genai.types.GenerateContentConfig(temperature=0.7)
            )
            ai_reminder = response.text.strip()
        except Exception as e:
            ai_reminder = f"❌ AI 提醒生成失敗：{e} (請檢查 API Key)"
    else:
         ai_reminder = "❌ AI 客戶端未初始化，無法生成提醒。"

    # 輸出結果與回寫 (加分任務: 自動建立分頁記錄摘要)
    WEEKLY_SUMMARY_TITLE = f'{target_weekday}課程摘要'

    try:
        worksheet_summary = sh_result.worksheet(WEEKLY_SUMMARY_TITLE)
    except gspread.WorksheetNotFound:
        worksheet_summary = sh_result.add_worksheet(title=WEEKLY_SUMMARY_TITLE, rows="100", cols="5")

    summary_data = [
        ["日期", "星期", "課程總覽", "本週重點總結", "AI 行前提醒"],
        [pd.Timestamp.now().strftime('%Y-%m-%d'), target_weekday, formatted_schedule, all_notes, ai_reminder]
    ]

    worksheet_summary.update(summary_data, value_input_option='USER_ENTERED')

    output_ai_reminder = f"**🤖 AI 行前提醒：**\n{ai_reminder}"
    output_summary = f"✅ 已將本次 {target_weekday} 的課程摘要回寫至 Google Sheet 的 **{WEEKLY_SUMMARY_TITLE}** 分頁。"

    return output_summary, f"--- {target_weekday} 課程清單 ---", formatted_schedule, "---", output_ai_reminder

# --- Gradio 輔助函式: 載入課程列表 ---
def load_course_list():
    """載入所有不重複的課程名稱供下拉選單使用"""
    _, df_schedule = initialize_gspread_and_data()
    if df_schedule is not None:
        return df_schedule[COL_COURSE].unique().tolist()
    return []

In [28]:
import gradio as gr

# 嘗試初始化並取得課程列表 (用於 Gradio 啟動時的預設值)
initial_status, df_initial = initialize_gspread_and_data()
initial_courses = df_initial[COL_COURSE].unique().tolist() if df_initial is not None else []

with gr.Blocks(title="作業六：課表查詢與 AI 提醒") as demo:
    gr.Markdown("# 📅 作業六：課表查詢與 AI 提醒生成 (Gradio 互動介面)")
    gr.Markdown("此工具支援：1. 手動更新/清空課程重點，2. 查詢指定日課程並生成 AI 行前提醒，3. 自動回寫結果到 Google Sheet。")

    gr.Markdown(f"--- Google Sheet 連線狀態：{initial_status} ---")

    with gr.Tab("📝 1. 手動更新/清空課程提醒"):
        gr.Markdown("### 請選擇課程，您可以輸入新重點，或點擊『清空』按鈕來覆蓋/刷新舊重點。")

        course_dropdown_value = initial_courses[0] if initial_courses else "無課程資料"

        course_dropdown = gr.Dropdown(
            label="選擇課程名稱",
            choices=initial_courses,
            value=course_dropdown_value
        )
        reminder_input = gr.Textbox(label="輸入本週重點 (可輸入空字串來清空)", placeholder="請輸入本週重點...")

        with gr.Row():
            update_btn = gr.Button("🔄 確認輸入新重點並回寫")
            clear_note_btn = gr.Button("🗑️ 清空此課程重點 (覆蓋刷新)")

        update_output = gr.Markdown("等待更新...")

        # 綁定「更新」函式
        update_btn.click(
            update_reminder_and_writeback,
            inputs=[course_dropdown, reminder_input],
            outputs=[update_output, course_dropdown]
        )

        # 綁定「清空」函式：傳入空字串 ''
        clear_note_btn.click(
            lambda c: update_reminder_and_writeback(c, ''),
            inputs=[course_dropdown],
            outputs=[update_output, course_dropdown]
        )

    with gr.Tab("🔍 2. 查詢當日課程與 AI 提醒"):
        gr.Markdown("### 選擇星期幾，查詢當日課程細節，並生成 AI 行前提醒。")

        weekday_input = gr.Radio(
            label="選擇查詢的星期",
            choices=["一", "二", "三", "四", "五", "六", "日"],
            value="四"
        )

        query_btn = gr.Button("🚀 查詢課程並生成 AI 提醒")

        gr.Markdown("### 查詢結果：")
        summary_output = gr.Markdown(label="回寫狀態", value="")
        title_courses = gr.Markdown(label="當日課程標題", value="")
        course_details_output = gr.Markdown(label="課程細節 (已包含本週重點)", value="")
        separator = gr.Markdown(label="", value="")
        ai_reminder_output = gr.Markdown(label="AI 行前提醒", value="")

        # 綁定查詢與 AI 生成函式
        query_btn.click(
            generate_daily_reminder,
            inputs=[weekday_input],
            outputs=[summary_output, title_courses, course_details_output, separator, ai_reminder_output]
        )

# 啟動 Gradio 應用
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dee568d62fb232ebb8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
